# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/datasets/data.csv')
display(data)
data.info() #столбцы 'days_employed' и 'total_income' имеют отличные от других столбцов значения
data.isna().sum()
#2174 пропущенных значений в столбцах 'days_employed' и 'total_income' (ожидаемые пропущенные значения - NaN)
#Данные пропуски - пропуски в количественных переменных, поэтому заменим их характерными значениями

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

Как мы видим, пропуски есть в столбцах 'days_emploted' и 'total_income'. Прежде чем начинать работать с данными, нужно заполнить данные пропуски

## Шаг 2. Предобработка данных

### Обработка пропусков

При первом ознакомлении заметно, что данные в столбце 'days_employed' встречаются как положительные, так и отрицательные, поэтому рассмотрим их по порядку



In [2]:
#Положительные значения:
data[data['days_employed'] > 0]['days_employed'].min()

328728.72060451825

328728 дней - это более 900 лет - многовато для минимального значения стажа (среди положительных значений). Значит здесь какая-то ошибка. Предположим, что это связано с тем, что считались не дни, а часы: значит, положительные значения в данном столбце нужно разделить на 24

In [3]:
data.loc[data['days_employed'] > 0, 'days_employed'] /= 24
data['days_employed'].max()

16739.80835313875

16749 дней - это почти 46 лет - звучит уже лучше. Будем обрабатывать значения дальше, на очереди отрицательные.

Как мы понимаем, количество дней отрицательным быть не может, значит ошибка в том, как эти дни считаются (то есть в системе подсчета). Для начала попробуем взять эти значения по модулю и поссмотреть, что из этого выйдет: 

In [4]:
#Отрицательные значения (могли появиться вследствие иной системы подсчета часов) возьмем по модулю:
abs(data[data['days_employed'] < 0]['days_employed'].min())

18388.949900568383

18388 дней - 50 с небольшим лет - максимальный стаж (среди отрицательных значений). В это верится больше, но у данных значений нужно поменять знак

In [5]:
data.loc[data['days_employed'] < 0, 'days_employed'] *= (-1)

Теперь найдем минимальное и максимальное значения в обновленных данных:

In [6]:
print('Минимальное значение стажа: {0} дней\nМаксимальное значение стажа: {1} дней = {2:.0f} лет'.format(data['days_employed'].min(), data['days_employed'].max(), data['days_employed'].max() / 365))

Минимальное значение стажа: 24.14163324048118 дней
Максимальное значение стажа: 18388.949900568383 дней = 50 лет


18388 дней или 50 лет стажа - звучит более правдоподобно

Прежде чем продолжить разбираться с пропусками в столбце 'days_employed', рассмотрим значения в столбце 'dob_years' (возраст клиента)

In [7]:
data['dob_years'].value_counts() #как мы видим, в наших данных встречаются клиенты с возрастом 0 аж 101 раз

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Рассмотрим строчки, в которых значение возраста равно 0. Возможно заметим в них какую-то связь:

In [8]:
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Данные идут в разнобой: 'ноль' стоит как у пенсионеров, так и у сотрудников; как у женщин, так и у мужчин; у представителей разного уровня образований - то куда относить этот 'ноль' непонятно, так что удалим данные строчки.

In [9]:
data = data.drop(data.index[data['dob_years'] == 0])

Вернемся к заполнению пропусков в столбце 'days_employed'

Создадим столбец с показателем количества отработанных дней за год в среднем для каждого заемщика

In [10]:
data['days_employed_per_year'] = data['days_employed'] / (data['dob_years'] - 16) #по законам РФ официальное трудоустройство возможно только с 16-ти лет
data[data['days_employed_per_year'] > 366].sort_values('days_employed_per_year')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
13029,0,16104.423292,60,среднее,1,женат / замужем,0,F,пенсионер,0,59773.456711,покупка жилья,366.009620
20253,0,15009.995825,57,ВЫСШЕЕ,0,женат / замужем,0,F,пенсионер,0,73792.951395,сделка с подержанным автомобилем,366.097459
8337,0,15011.160203,57,среднее,1,вдовец / вдова,2,M,пенсионер,0,82876.335652,строительство жилой недвижимости,366.125859
19712,0,16113.315481,60,среднее,1,женат / замужем,0,F,пенсионер,0,48346.184126,профильное образование,366.211715
2718,0,15747.576181,59,среднее,1,женат / замужем,0,F,пенсионер,0,88749.933619,образование,366.222702
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13953,0,15701.024409,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем,1427.365855
12507,0,15812.170938,27,среднее,1,Не женат / не замужем,4,F,пенсионер,0,50969.340462,сделка с автомобилем,1437.470085
16166,0,15181.174890,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль,1518.117489
19439,0,16224.881982,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости,1622.488198


Как мы видим, в 1340 строках средний стаж в год больше, чем 366, что невозможно. Посмотрим на данные, может удастся найти закономерность в строчках с неверными значениями

In [11]:
data[data['days_employed_per_year'] > 366]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,383.182514
18,0,16678.380705,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,450.767046
87,0,14069.147292,53,среднее,1,женат / замужем,0,F,пенсионер,0,74363.856554,покупка жилья для сдачи,380.247224
98,0,15204.425239,54,высшее,0,женат / замужем,0,F,пенсионер,0,199707.298524,покупка жилья для сдачи,400.116454
140,0,3017.782065,24,среднее,1,гражданский брак,1,F,сотрудник,0,102110.989149,покупка коммерческой недвижимости,377.222758
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21378,0,16719.031045,59,среднее,1,вдовец / вдова,2,F,пенсионер,0,204103.704870,операции со своей недвижимостью,388.814675
21398,0,14921.079975,55,СРЕДНЕЕ,1,Не женат / не замужем,4,F,пенсионер,0,89194.369579,строительство жилой недвижимости,382.591794
21433,0,16280.502364,57,среднее,1,гражданский брак,1,F,пенсионер,0,139742.106467,на проведение свадьбы,397.085424
21475,0,15217.221094,52,среднее,1,женат / замужем,0,F,пенсионер,1,48239.310903,свой автомобиль,422.700586


К сожалению, выявить какую-то взаимосвязь не удалось, то есть исправить данные не получится, следственно, данные строчки придется удалить

In [12]:
data = data.drop(data.index[data['days_employed_per_year'] > 366])

Вычислим среднее количество дней стажа за год тех заемщиков, у которых данный показатель <366

In [13]:
avg_days_employed_per_year = data[data['days_employed_per_year'] <= 366]['days_employed_per_year'].median()

Заполним пропуски в столбце 'days_employed' характерным значением - медиана среднего количсества дней стажа, накапливающихся за год, умноженная на количество лет с момента возможности официального трудоустройства

In [14]:
data.loc[data['days_employed'].isna(), 'days_employed'] = avg_days_employed_per_year * (data['dob_years'] - 16)
data['days_employed'].isna().sum() #пропуски в столбце 'days_employed' заполнены

0

Попробуем найти взаимосвязь между уровнем дохода и различными характеристиками заемщиков (количеством детей в семье, уровнем образования, семейным положением, полом и статусом работы)

In [15]:
data.pivot_table(
    index='children',
    values='total_income',
    aggfunc='median'
)

,total_income
children,
-1,146253.325480
0,145900.544012
1,148388.755249
2,144650.291209
3,157447.617463
4,157568.585593
5,186351.410251
20,145334.865002


In [17]:
data.pivot_table(
    index='family_status_id',
    values='total_income',
    aggfunc='median'
)

,total_income
family_status_id,
0,147849.624770
1,146752.793602
2,128540.213683
3,150150.119927
4,145903.057821


In [18]:
data.pivot_table(
    index='gender',
    values='total_income',
    aggfunc='median'
)

,total_income
gender,
F,135449.561416
M,168739.412523
XNA,203905.157261


In [19]:
data.pivot_table(
    index='income_type',
    values='total_income',
    aggfunc='median'
)

,total_income
income_type,
в декрете,53829.130729
госслужащий,150521.917580
компаньон,172396.000846
пенсионер,115413.406988
предприниматель,499163.144947
сотрудник,142607.772745
студент,98201.625314


Заполнять пропущенные данные в столбце заработка средним значением по всем клиентам будет неверно, так как, как мы видим, доходы среди разных категорий заемщиков по типу занятости могут сильно отличаться. Например, средний предприниматель в месяц зарабатывает почти 500 тысяч рублей, тогда как заемщик в декрете - всего порядка 50 тысяч.

Также были замечены странные значения в столбцах 'children' и 'gender', но к ним мы еще вернемся, а пока...

Заполним пропуски в соответствие с категорией занятости данного сотрудника

In [20]:
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'в декрете'), 'total_income'] = data[data['income_type'] == 'в декрете']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'госслужащий'), 'total_income'] = data[data['income_type'] == 'госслужащий']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'компаньон'), 'total_income'] = data[data['income_type'] == 'компаньон']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'пенсионер'), 'total_income'] = data[data['income_type'] == 'пенсионер']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'предприниматель'), 'total_income'] = data[data['income_type'] == 'предприниматель']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'сотрудник'), 'total_income'] = data[data['income_type'] == 'сотрудник']['total_income'].median()
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'студент'), 'total_income'] = data[data['income_type'] == 'студент']['total_income'].median()

Проверим, не осталось ли у нас незаполненных значений в столбце с месячным заработком

In [21]:
data['total_income'].isna().sum() #пропуски в столбце 'total_income' заполнены

0

Рассмотрим те самые странные значения в столбце 'children'

In [22]:
data['children'].value_counts()

 0     12908
 1      4666
 2      2024
 3       322
 20       72
-1        43
 4        40
 5         9
Name: children, dtype: int64

Видим некорректное значение -1 и странное значение 20, притом что максимальное до него 5.
Можем предположить, что вместо -1 должно быть 1, а вместо 20 - 2. Давайте посмотрим на строчки с каждым из этих значений по отдельности

In [23]:
data[data['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование,147.256786
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля,26.531898
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование,77.425761
941,-1,3870.017071,57,Среднее,1,женат / замужем,0,F,пенсионер,0,115413.406988,на покупку своего автомобиля,NaN
1363,-1,1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование,30.647819
1929,-1,1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья,66.422879
2073,-1,2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья,97.683124
3814,-1,3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы,304.529044
4201,-1,901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью,36.044070
4402,-1,16583.387620,64,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,163264.062064,покупка недвижимости,345.487242


In [24]:
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья,176.044223
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости,30.556983
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования,82.760290
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью,63.120029
3302,20,1793.422545,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,150521.917580,профильное образование,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль,51.677413
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование,28.627375
21390,20,3492.454430,53,среднее,1,женат / замужем,0,M,компаньон,0,172396.000846,покупка жилой недвижимости,NaN
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью,13.744124


По значению -1 все понятно: такого не может быть, в принципе. Так как все клиенты совершеннолетние, то найти причины, по которым у любого из них не может быть 1 ребенок мы не можем, так что заменим -1 на 1.

С клиентами, у которых отметка на 20 детей ситуация чуть сложнее: с одной стороны, есть клиенты, которые в силу возраста не могли бы иметь столько детей (например, 21-летний молодой человек с 20 детьми - звучит фантастически); с другой стороны есть клиенты в возрасте, которые чисто теоретически могли обзавестись таким количеством детей, но отталкиваясь от того, что таких клиентов в нашей базе получилось почти в 2 раза больше, чем клиентов с 4 детьми, и в 8 раз больше, чем клиентов с 5 детьми, а также от того факта, что клиентов с количеством детей от 6 до 19 вообще нет, можно понять, что здесь что-то не так. Так что этой группе заемщиков мы тоже нашли новый дом

Заменим значения -1 на 1, а 20 - на 2

In [25]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

Проверяем - все ли заменилось успешно?

In [26]:
data['children'].value_counts()

0    12908
1     4709
2     2096
3      322
4       40
5        9
Name: children, dtype: int64

Да, теперь все выглядит красиво и логично!

Проверим общую информацию о данных с помощью метода .info(), а также подсчитаем количество незаполненных ячеек связки методов .isna() и .sum()

In [27]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20084 entries, 0 to 21524
Data columns (total 13 columns):
children                  20084 non-null int64
days_employed             20084 non-null float64
dob_years                 20084 non-null int64
education                 20084 non-null object
education_id              20084 non-null int64
family_status             20084 non-null object
family_status_id          20084 non-null int64
gender                    20084 non-null object
income_type               20084 non-null object
debt                      20084 non-null int64
total_income              20084 non-null float64
purpose                   20084 non-null object
days_employed_per_year    17920 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.8+ MB


children                     0
days_employed                0
dob_years                    0
education                    0
education_id                 0
family_status                0
family_status_id             0
gender                       0
income_type                  0
debt                         0
total_income                 0
purpose                      0
days_employed_per_year    2164
dtype: int64

**Вывод**

Пропуски в столбцах 'days_employed' и 'total_income' были заполнены характерными значениями: в первом случае это медиана дней стажа в год, уможенная на количество лет трудоспособного возраста, а во втором случае - медиана из зарплат по типу занятости заемщика. Данные пропуски, скорее всего, связаны с технологических сбоями.

Также были проанализированы остальные столбцы с данными и сделаны следующие изменения: значения '-1' и '20' в столбце 'children' заменены на '1' и '2', соответственно; строчки со значением '0' в столбце 'dob_years' - удалены. Данные значения были немногочисленны, и, можно предположить, что они связаны с человеческим фактором (опечатки и прочее)

Все строчки в столбце 'days_employed' исправлены на корректные (положительные поделены на '24', отрицательные - изменили на положительные). Связываем это с тем, что данные приходили с разных источников: с одного - с правильным знаком, но в часах, вместо дней; а со второго - в правильной единице измерения, но с противоположным знаком.

### Замена типа данных

Приведем данные столбцов 'days_employed' и 'total_income' к целочисленному типу данных

In [28]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info() #проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20084 entries, 0 to 21524
Data columns (total 13 columns):
children                  20084 non-null int64
days_employed             20084 non-null int64
dob_years                 20084 non-null int64
education                 20084 non-null object
education_id              20084 non-null int64
family_status             20084 non-null object
family_status_id          20084 non-null int64
gender                    20084 non-null object
income_type               20084 non-null object
debt                      20084 non-null int64
total_income              20084 non-null int64
purpose                   20084 non-null object
days_employed_per_year    17920 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.8+ MB


**Вывод**

Данные в столбцах 'days_employed' и 'total_income' приведены к целочисленному типу данных, что будет затрачивать меньше памяти при вычислениях

### Обработка дубликатов

Сначала найдем грубые дубликаты

In [29]:
data.duplicated().sum()

54

54 абсолютно одинаковых строчек - звучит не очень - но надо посмотреть на них повнимательней

In [30]:
data[(data.duplicated() == True)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
2849,0,2359,41,среднее,1,женат / замужем,0,F,сотрудник,0,142607,покупка жилья для семьи,NaN
4182,1,1699,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142607,свадьба,NaN
4851,0,4153,60,среднее,1,гражданский брак,1,F,пенсионер,0,115413,свадьба,NaN
5557,0,3964,58,среднее,1,гражданский брак,1,F,пенсионер,0,115413,сыграть свадьбу,NaN
7808,0,3870,57,среднее,1,гражданский брак,1,F,пенсионер,0,115413,на проведение свадьбы,NaN
8583,0,3964,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,115413,дополнительное образование,NaN
9238,2,1699,34,среднее,1,женат / замужем,0,F,сотрудник,0,142607,покупка жилья для сдачи,NaN
9528,0,4719,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,115413,операции со своей недвижимостью,NaN
9627,0,3775,56,среднее,1,женат / замужем,0,F,пенсионер,0,115413,операции со своей недвижимостью,NaN
10462,0,4341,62,среднее,1,женат / замужем,0,F,пенсионер,0,115413,покупка коммерческой недвижимости,NaN


In [31]:
data[(data.duplicated() == True)]['total_income'].value_counts()

142607    28
115413    19
172396     5
150521     2
Name: total_income, dtype: int64

Как мы видим, все значения заработков являются теми медианами для разных типов занятости, которыми мы заполняли пропуски... Давайте разбираться дальше.

Если не брать в учет совпадения значений в данном столбце, то какова вероятность того, что совпадут значения в других?

P = Pchildren * Pdob_years * Peducation_id * Pfamily_status * Pgender * Pdebt * Ppurpose = (посчитаем грубо) 1/8 * 1/50 * 1/5 * 1/5 * 1/2 * 1/2 * 1/0 = 1/1600000 - вероятность появления 1 дубликата

Но так как у нас около 20 тысяч клиентов, то вероятность того, что среди 20 тысяч найдутся 54 человека с абсолютно одинаковыми данными равна: 

1/1600000 * 20000 / 54 = 0.00023 - чуть больше, чем 0.02%

Вероятность того, что в наших данных 54 клиента с абсолютно одинаковыми анкетами крайне мала. В совпадения и мистику мы не верим, поэтому со спокойной душой удалим грубые дубликаты

In [32]:
data = data.drop_duplicates().reset_index(drop=True)

И сразу сделаем проверку

In [33]:
data.duplicated().sum()

0

Как видно, грубые дубликаты исчезли, дальше будем работать без них

С помощью функции value_counts() посмотрим уникальные значения в строках с качественными переменными

In [34]:
data['education'].value_counts()

среднее                12674
высшее                  4486
СРЕДНЕЕ                  715
Среднее                  651
неоконченное высшее      651
ВЫСШЕЕ                   260
Высшее                   254
начальное                230
Неоконченное высшее       45
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 15
Начальное                 14
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В данном столбце видно, что встречаются дубликаты, различные лишь регистром написания ('СРЕДНЕЕ' и 'среднее').
Переведем весь столбец в нижний регистр

In [35]:
data['education'] = data['education'].str.lower()
data['education'].value_counts() #проверка - дубликаты ушли

среднее                14040
высшее                  5000
неоконченное высшее      725
начальное                259
ученая степень             6
Name: education, dtype: int64

Проделаем тоже самое для столбца 'family_status'

In [36]:
data['family_status'].value_counts()

женат / замужем          11530
гражданский брак          3903
Не женат / не замужем     2669
в разводе                 1105
вдовец / вдова             823
Name: family_status, dtype: int64

 Дубликатов нет, но одно значение начинается на заглавную букву - заменим ее на строчную

In [37]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts() #проверка - теперь заглавной буквы нет

женат / замужем          11530
гражданский брак          3903
не женат / не замужем     2669
в разводе                 1105
вдовец / вдова             823
Name: family_status, dtype: int64

...И тоже самое со столбцом 'gender'...

In [38]:
data['gender'].value_counts()

F      13021
M       7008
XNA        1
Name: gender, dtype: int64

Дубликатов нет, но есть одно некорректное значение 'XNA' - видимо, это обозначение 'не определено'. Удалим это значение

In [39]:
data = data.drop(data.index[data['gender'] == 'XNA'])
data['gender'].value_counts() #проверка

F    13021
M     7008
Name: gender, dtype: int64

Также рассмотрим столбец 'income_type'

In [40]:
data['income_type'].value_counts()

сотрудник          10972
компаньон           5046
пенсионер           2568
госслужащий         1439
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Дубликаты не найдены, но найдены редкие значения 'предприниматель', 'в декрете' и 'студент' - что с ними делать - непонятно. Хочется присоединить к другим, но оставлю это на усмотрение коллег 

После обработки столбцов с категориальными переменными, проверим нашу таблицу на грубые дубликаты еще раз

In [41]:
data.duplicated().sum()

17

В результате обработки категориальных переменных и уменьшению количества этих категорий в разных столбцах появилась новая порция грубых дубликатов - избавимся и от них

In [42]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Ура! Все дубликаты успешно ушли, можно двигаться дальше!

**Вывод**

Дубликаты были обнаружены только в столбце 'education'. Все они были из-за написания в разном регистре. После приведения всех значения к одному (нижнему) регистру, количество ункальных значений в столбце сократилось, дубликаты исчезли.

Также были обнаружены грубые дубликаты, которые возникли в результате заполнения столбца 'total_income' одинаковым характерным значением, а также совпадением значений в остальных столбцах. Такие дубликаты мы удалять не стали

### Лемматизация

Импортируем необходимые библиотеки и контейнеры

In [43]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Рассмотрим уникальные значения в столбце 'purpose' и их количество

In [44]:
data['purpose'].value_counts()

свадьба                                   745
сыграть свадьбу                           713
на проведение свадьбы                     712
покупка коммерческой недвижимости         624
операции с недвижимостью                  623
операции с коммерческой недвижимостью     604
операции с жильем                         600
покупка жилья для семьи                   600
покупка жилья                             599
жилье                                     597
недвижимость                              597
покупка жилья для сдачи                   596
операции со своей недвижимостью           592
строительство собственной недвижимости    592
покупка своего жилья                      582
строительство недвижимости                582
строительство жилой недвижимости          578
покупка жилой недвижимости                568
покупка недвижимости                      566
ремонт жилью                              566
на покупку своего автомобиля              479
заняться высшим образованием      

Запишем в переменную 'purpose_unique' уникальные значения из столбца 'purpose'

In [45]:
purpose_unique = data['purpose'].unique()

Разобьем цели на отдельные слова

In [46]:
lemmas = []
for str in purpose_unique: 
    lemmas += [x for x in m.lemmatize(str) if x != '\n' and x != ' ']
    
print(Counter(lemmas))

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'проведение': 1, 'семья': 1, 'сыграть': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Выберем из самых часто встречающихся терминов те, что еще и информативные: 'недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба' - и создадим функцию для лемматизации данных

In [47]:
def what_is_purpose(purpose): #функция для распределения целей кредита на категории по ключевым словам
    lemm = m.lemmatize(purpose)
    if 'жилье' in lemm: 
        return 'недвижимость'
    if 'недвижимость' in lemm:
        return 'недвижимость'
    if 'автомобиль' in lemm:
        return 'автомобиль'
    if 'свадьба' in lemm:
        return 'свадьба'
    if 'образование' in lemm:
        return 'образование'

Применим данную функцию к набору данных и проверим количество уникальных значений в столбце теперь

In [48]:
data['purpose'] = data['purpose'].apply(what_is_purpose)   
data['purpose'].value_counts()

недвижимость    10066
автомобиль       4030
образование      3746
свадьба          2170
Name: purpose, dtype: int64

Все круто!

**Вывод**

Мы рассмотрели всевозможные цели кредита и с помощью лемматизации выделили 4 общих категории: "недвижимость", "автомобиль", "образование" и "свадьба". Как мы видим: категория "недвижимость" с большим отрывом на первом месте - больше половины кредитов; меньше всего кредиты берут на свадьбу - всего 2 тысячи - десятая часть от всех кредитов

### Категоризация данных

Для начала посмотрим на табличку, которую мы получили к этому моменту, и оценим данные, которые у нас есть

In [49]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,324.525886
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,201.240188
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,330.789565
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,257.796700
4,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,84.198712


Категоризация данных нам необходима для тех случаев, когда данные количественные и, соответственно, могут принимать случайные
значения. В нашей таблице есть три столбца, которые так и хочется категоризировать (потому что в данных столбцах разнообразие
значений больше, чем в остальных столбцах) - это 'days_employed', 'dob_years' и 'total_income'. Напишем функции, которые
на основании данных будут выделять 2-4 категории заемщиков. Применим для нашей таблицы

In [50]:
def days_employed_category(days): #функция категоризации по стажу
    if days < 365:
        return 'меньше года'
    if 365 <= days < 730 :
        return 'от 1 до 2 лет'
    if 730 <= days < 1825 :
        return 'от 2 до 5 лет'
    if days >= 1825:
        return 'больше 5 лет'

In [51]:
def dob_years_category(years): #функция категоризации по возрасту
    if years < 20:
        return '<20'
    if 20 <= years < 30:
        return '20-30'
    if 30 <= years < 40:
        return '30-40'
    if 40 <= years < 50:
        return '40-50'
    if 50 <= years < 60:
        return '50-60'
    if years >= 60:
        return '60+'

In [52]:
def total_income_category(salary): #функция категоризации по доходу
    if salary < 100000:
        return '< 100 т.р.'
    if 100000 <= salary < 200000 :
        return '100 - 200 т.р.'
    if salary >= 200000:
        return '> 200 т.р.'

Применим данные функции на наш набор данных и посмотрим на результат их работы

In [53]:
data['days_employed_category'] = data['days_employed'].apply(days_employed_category)
data['dob_years_category'] = data['dob_years'].apply(dob_years_category)
data['total_income_category'] = data['total_income'].apply(total_income_category)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year,days_employed_category,dob_years_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,324.525886,больше 5 лет,40-50,> 200 т.р.
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,201.240188,больше 5 лет,30-40,100 - 200 т.р.
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,330.789565,больше 5 лет,30-40,100 - 200 т.р.
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,257.796700,больше 5 лет,30-40,> 200 т.р.
4,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,84.198712,от 2 до 5 лет,20-30,> 200 т.р.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость,167.752469,больше 5 лет,40-50,> 200 т.р.
20008,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль,280.994611,больше 5 лет,60+,100 - 200 т.р.
20009,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,96.061222,больше 5 лет,30-40,< 100 т.р.
20010,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль,141.476441,больше 5 лет,30-40,> 200 т.р.


Совсем другое дело!

Теперь посмотрим, как распределяются клиенты, имевшие задолжность, для некоторых категорий

In [54]:
#Категория "количество детей"
children_0_total = data[(data['children'] == 0)]['debt'].count() #общее количество заемщиков с 0 детей
children_0_debtors = data[(data['children'] == 0) & (data['debt'] == 1)]['debt'].count() #количество заемщиков с 0 детьми, просрочивших кредит
children_0_debtors_share = children_0_debtors / children_0_total #доля заемщиков с 0 детьми, просрочившими кредит, от общего числа заемщиков с 0 детьми

children_1_total = data[(data['children'] == 1)]['debt'].count() #здесь и далее аналогично с предыдущим пунктом
children_1_debtors = data[(data['children'] == 1) & (data['debt'] == 1)]['debt'].count()
children_1_debtors_share = children_1_debtors / children_1_total

children_2_total = data[(data['children'] == 2)]['debt'].count()
children_2_debtors = data[(data['children'] == 2) & (data['debt'] == 1)]['debt'].count()
children_2_debtors_share = children_2_debtors / children_2_total

children_3_total = data[(data['children'] >= 3)]['debt'].count()
children_3_debtors = data[(data['children'] >= 3) & (data['debt'] == 1)]['debt'].count()
children_3_debtors_share = children_3_debtors / children_3_total

На основе нашего анализа создадим новую таблицу с данными, отражающую надежность клиента в зависимости от количества детей

In [55]:
debtors_by_children_data = [['0', children_0_debtors, children_0_total, children_0_debtors_share],
                            ['1', children_1_debtors, children_1_total, children_1_debtors_share],
                            ['2', children_2_debtors, children_2_total, children_2_debtors_share],
                            ['3+', children_3_debtors, children_3_total, children_3_debtors_share]]
debtors_by_children_columns = ['number_of_children', 'number_of_debtors', 'total', 'share_of_debtors']
debtors_by_children = pd.DataFrame(debtors_by_children_data, columns = debtors_by_children_columns)
debtors_by_children

,number_of_children,number_of_debtors,total,share_of_debtors
0,0,1001,12849,0.077905
1,1,433,4699,0.092147
2,2,199,2093,0.095079
3,3+,30,371,0.080863


Как мы видим, доля должников относительно количества детей пропорционально возрастает. Доля "должников" при количестве детей более трех может быть не совсем корректна, так как выборка клиентов небольшая

In [56]:
#Категория "семейное положение"
family_status_0_total = data[(data['family_status_id'] == 0)]['debt'].count() #аналогично с предыдущим пунктом
family_status_0_debtors = data[(data['family_status_id'] == 0) & (data['debt'] == 1)]['debt'].count()
family_status_0_debtors_share = family_status_0_debtors / family_status_0_total

family_status_1_total = data[(data['family_status_id'] == 1)]['debt'].count()
family_status_1_debtors = data[(data['family_status_id'] == 1) & (data['debt'] == 1)]['debt'].count()
family_status_1_debtors_share = family_status_1_debtors / family_status_1_total

family_status_2_total = data[(data['family_status_id'] == 2)]['debt'].count()
family_status_2_debtors = data[(data['family_status_id'] == 2) & (data['debt'] == 1)]['debt'].count()
family_status_2_debtors_share = family_status_2_debtors / family_status_2_total

family_status_3_total = data[(data['family_status_id'] == 3)]['debt'].count()
family_status_3_debtors = data[(data['family_status_id'] == 3) & (data['debt'] == 1)]['debt'].count()
family_status_3_debtors_share = family_status_3_debtors / family_status_3_total

family_status_4_total = data[(data['family_status_id'] == 4)]['debt'].count()
family_status_4_debtors = data[(data['family_status_id'] == 4) & (data['debt'] == 1)]['debt'].count()
family_status_4_debtors_share = family_status_4_debtors / family_status_4_total

Создадим новую таблицу с данными, показывающую надежность клиента в зависимости от его семейного положения

In [57]:
debtors_by_family_status_data = [['женат / замужем', family_status_0_debtors, family_status_0_total, family_status_0_debtors_share],
                                ['гражданский брак', family_status_1_debtors, family_status_1_total, family_status_1_debtors_share],
                                ['вдовец / вдова', family_status_2_debtors, family_status_2_total, family_status_2_debtors_share],
                                ['в разводе', family_status_3_debtors, family_status_3_total, family_status_3_debtors_share],
                                ['не женат / не замужем', family_status_4_debtors, family_status_4_total, family_status_4_debtors_share]]
debtors_by_family_status_columns = ['family_status', 'number_of_debtors', 'total', 'share_of_debtors']
debtors_by_family_status = pd.DataFrame(debtors_by_family_status_data, columns = debtors_by_family_status_columns)
debtors_by_family_status.sort_values('share_of_debtors')

,family_status,number_of_debtors,total,share_of_debtors
2,вдовец / вдова,55,823,0.066829
3,в разводе,79,1105,0.071493
0,женат / замужем,889,11525,0.077137
1,гражданский брак,370,3890,0.095116
4,не женат / не замужем,270,2669,0.101161


Клиенты в браке должны вызывать больше доверия, чем клиенты, состоящие в гражданском браке и не состоящие в браке вообще. Удивительны доли вдовцов и разведенных людей, не погасивших кредит в срок, - у них они самые низкие

In [58]:
#Категория "уровень дохода"
salary_0_total = data[(data['total_income_category'] == '< 100 т.р.')]['debt'].count() #рутина-рутина-рутина
salary_0_debtors = data[(data['total_income_category'] == '< 100 т.р.') & (data['debt'] == 1)]['debt'].count()
salary_0_debtors_share = salary_0_debtors / salary_0_total

salary_1_total = data[(data['total_income_category'] == '100 - 200 т.р.')]['debt'].count()
salary_1_debtors = data[(data['total_income_category'] == '100 - 200 т.р.') & (data['debt'] == 1)]['debt'].count()
salary_1_debtors_share = salary_1_debtors / salary_1_total

salary_2_total = data[(data['total_income_category'] == '> 200 т.р.')]['debt'].count()
salary_2_debtors = data[(data['total_income_category'] == '> 200 т.р.') & (data['debt'] == 1)]['debt'].count()
salary_2_debtors_share = salary_2_debtors / salary_2_total

Создадим еще одну таблицу, демонстрирующую надежность клиента относительно его ежемесячного дохода

In [59]:
debtors_by_salary_data = [['< 100 т.р.', salary_0_debtors, salary_0_total, salary_0_debtors_share],
                          ['100 - 200 т.р.', salary_1_debtors, salary_1_total, salary_1_debtors_share],
                          ['> 200 т.р.', salary_2_debtors, salary_2_total, salary_2_debtors_share]]
debtors_by_salary_columns = ['salary', 'number_of_debtors', 'total', 'share_of_debtors']
debtors_by_salary = pd.DataFrame(debtors_by_salary_data, columns = debtors_by_salary_columns)
debtors_by_salary

,salary,number_of_debtors,total,share_of_debtors
0,< 100 т.р.,336,3988,0.084253
1,100 - 200 т.р.,982,11230,0.087444
2,> 200 т.р.,345,4794,0.071965


In [60]:
#Категория "уровень дохода" через pivot_table() и join()
number_of_debtors_by_total_income = data[data['debt'] == 1].pivot_table(
    index='total_income_category',
    values='debt',
    aggfunc='count'
)
number_of_debtors_by_total_income.columns = ['number_of_debtors']

total_clients_by_total_income = data.pivot_table(
    index='total_income_category',
    values='total_income',
    aggfunc='count'
)
total_clients_by_total_income.columns = ['total']

share_of_debtors_by_total_income = data.pivot_table(
    index='total_income_category',
    values='debt',
    aggfunc='mean'
)
share_of_debtors_by_total_income.columns = ['share_of_debtors']

#display(number_of_debtors_by_total_income, total_clients_by_total_income, share_of_debtors_by_total_income)
final_share_of_debtors_by_total_income = number_of_debtors_by_total_income.join([total_clients_by_total_income,share_of_debtors_by_total_income])
final_share_of_debtors_by_total_income

,number_of_debtors,total,share_of_debtors
total_income_category,,,
100 - 200 т.р.,982,11230,0.087444
< 100 т.р.,336,3988,0.084253
> 200 т.р.,345,4794,0.071965


Доля клиентов, не погасивших кредит в срок начинает уменьшаться только с определенной суммы дохода в месяц (в районе 200 т.р. в месяц). Видимо, те клиенты, которые получают такую сумму денег имеют бОльшую финансовую подушку на случай непредвиденых обстоятельств и реже не имеют возможность внести оплату по кредиту

In [61]:
#Категория "цель кредита"
purpose_0_total = data[(data['purpose'] == 'недвижимость')]['debt'].count() #основано на реальных событиях... из предыдущего пункта
purpose_0_debtors = data[(data['purpose'] == 'недвижимость') & (data['debt'] == 1)]['debt'].count()
purpose_0_debtors_share = purpose_0_debtors / purpose_0_total

purpose_1_total = data[(data['purpose'] == 'автомобиль')]['debt'].count()
purpose_1_debtors = data[(data['purpose'] == 'автомобиль') & (data['debt'] == 1)]['debt'].count()
purpose_1_debtors_share = purpose_1_debtors / purpose_1_total

purpose_2_total = data[(data['purpose'] == 'образование')]['debt'].count()
purpose_2_debtors = data[(data['purpose'] == 'образование') & (data['debt'] == 1)]['debt'].count()
purpose_2_debtors_share = purpose_2_debtors / purpose_2_total

purpose_3_total = data[(data['purpose'] == 'свадьба')]['debt'].count()
purpose_3_debtors = data[(data['purpose'] == 'свадьба') & (data['debt'] == 1)]['debt'].count()
purpose_3_debtors_share = purpose_3_debtors / purpose_3_total

Создадим последную таблицу, в которой можно увидеть надежность клиента относительно целей его кредита

In [62]:
debtors_by_purpose_data = [['недвижимость', purpose_0_debtors, purpose_0_total, purpose_0_debtors_share],
                           ['автомобиль', purpose_1_debtors, purpose_1_total, purpose_1_debtors_share],
                           ['образование', purpose_2_debtors, purpose_2_total, purpose_2_debtors_share],
                           ['свадьба', purpose_3_debtors, purpose_3_total, purpose_3_debtors_share]]
debtors_by_purpose_columns = ['purpose', 'number_of_debtors', 'total', 'share_of_debtors']
debtors_by_purpose = pd.DataFrame(debtors_by_purpose_data, columns = debtors_by_purpose_columns)
print(debtors_by_purpose.sort_values('share_of_debtors'))

        purpose  number_of_debtors  total  share_of_debtors
0  недвижимость                749  10066          0.074409
3       свадьба                174   2170          0.080184
1    автомобиль                383   4030          0.095037
2   образование                357   3746          0.095302


Как мы видим, те заемщики, которые берут кредит на недвижимость и свадьбу, не погашают кредит в срок чуть реже, чем те, кто берет кредит на автомобиль и образование

**Вывод**

Мы категоризировали столбцы с количественными значениями, создав дополнительные столбцы с категориями.
По 4 категориям рассчитаны доли заемщиков, не погасивших кредит вовремя, приведены таблицы с показателями общего числа просрочивших кредит, общего числа клиентов, а также относительного числа клиентов, просрочивших кредит, ко всем клиентам по разным категориям. На основании проделанной работы можно делать следующие выводы...

## Шаг 3. Изучение зависимости задолженности от различных факторов

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [63]:
debtors_by_children

,number_of_children,number_of_debtors,total,share_of_debtors
0,0,1001,12849,0.077905
1,1,433,4699,0.092147
2,2,199,2093,0.095079
3,3+,30,371,0.080863


**Вывод**

Да, зависимость есть: чем меньше детей, тем выше вероятность того, что за кредит будет заплачено в срок (7.8% клиентов не оплачивают кредит в срок без детей, 9.2% - с одним ребенком и 9.5% - с двумя). Низкая доля клиентов, не оплативших кредит в срок с 3 и более количеством детей скорее говорит о том, что данных по многодетным семьям, недостаточно, чтобы делать точные выводы, нежели о тенденции к тому, что после второго ребенка клиенты становятся более аккуратны к выплатам по своим кредитам. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [64]:
debtors_by_family_status.sort_values('share_of_debtors')

,family_status,number_of_debtors,total,share_of_debtors
2,вдовец / вдова,55,823,0.066829
3,в разводе,79,1105,0.071493
0,женат / замужем,889,11525,0.077137
1,гражданский брак,370,3890,0.095116
4,не женат / не замужем,270,2669,0.101161


**Вывод**

Зависимость есть: клиенты, находящиеся в браке, значительно чаще возвращают кредит в срок (7.7%), нежели те, кто находится в гражданском браке (9.4%) или не женат/не замужем (10%). Удивительно, но те клиенты, что пережили развод или смерть супруга выплачивают кредит еще лучше - 7.1% и 6.6%, соответственно. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:
debtors_by_salary

,salary,number_of_debtors,total,share_of_debtors
0,< 100 т.р.,336,3988,0.084253
1,100 - 200 т.р.,982,11230,0.087444
2,> 200 т.р.,345,4794,0.071965


**Вывод**

Самыми надежными заемщиками по уровню дохода, очевидно, стали клиенты, чьи зарплаты выше, чем у большинства (более 200 тысяч рублей в месяц). Среди таких заемщиков лиш 7.2% не оплачивают кредит в срок. Среди тех, кто получает менее 200 тысяч рублей, - данная доля примерно равна 8.5%.

- Как разные цели кредита влияют на его возврат в срок?

In [66]:
debtors_by_purpose.sort_values('share_of_debtors')

,purpose,number_of_debtors,total,share_of_debtors
0,недвижимость,749,10066,0.074409
3,свадьба,174,2170,0.080184
1,автомобиль,383,4030,0.095037
2,образование,357,3746,0.095302


**Вывод**

Заемщики "на недвижимость" являются самыми надежными в категории "цель кредита" - они не выплачивают кредит только в 7.4% случаев, недалеко от них остались и те, кто оформляют кредит на свадьбу - 7.9%. Скорее всего, низкий процент в данных случаях говорит нам о том, что так или иначе вложения себя окупают (квартиру можно сдать, свадебные подарки зачастую носят материальный характер). В аутсайдерах по надежности в данной категории выступают те, кто берет кредит на автомобиль или образование - около 9.5% заемщиков не выплачивают кредит в срок.

## Шаг 4. Общий вывод

Идеальными для кредитного отдела банка клиентами будут являтся бездетные вдовцы с доходом свыше 200 тысяч рублей в месяц, берущие кредит на недвижимость. Давайте оценим, каковы вероятности, что в дверь зайдет именно такой клиент, а также что именно он не сможет в срок заплатить по кредиту:

In [67]:
perfect_clients_total = data[(data['children'] == 0) & (data['family_status_id'] == 2) & (data['total_income_category'] == '> 200 т.р.') & (data['purpose'] == 'недвижимость')]['debt'].count()
perfect_clients_debtors = data[(data['children'] == 0) & (data['family_status_id'] == 2) & (data['total_income_category'] == '> 200 т.р.') & (data['purpose'] == 'недвижимость') & (data['debt'] == 1)]['debt'].count()
clients_total = len(data)
perfect_clients_share = perfect_clients_total / clients_total
perfect_clients_debtors_share = perfect_clients_debtors / perfect_clients_total
print('Всего "идеальных" клиентов: {:.1%}\nОни не выплачивают кредит с вероятностью: {:.1%}'.format(perfect_clients_share, perfect_clients_debtors_share))

Всего "идеальных" клиентов: 0.3%
Они не выплачивают кредит с вероятностью: 4.6%


Как мы видим, вероятность первого события крайне мала, поэтому фокусироваться именно на данной группе клиентов будет неправильно. Нужно работать со всеми, но иметь в виду те риски, которые подсчитаны выше.

По итогам исследования мы выяснили, что чем вероятность выплаты кредита в срок:
- монотонно убывает относительно количества детей у клиента(по крайней мере, при количестве детей от 0 до 2);
- выше у людей, находящихся в законном браке(92.3%), либо имевших брак ранее - разведенных(92.9%) или овдовевших(93.3%) - по сравнению с теми, кто находится в гражданском браке(90.5%) или холост(90%);
- значительно повышается у категории людей, зарабатывающих более 200 тысяч рублей в месяц(92.8% против 91.6% и меньше при доходе менее 200 тысяч рублей);
- в среднем выше, если заемщик оформляет кредит на недвижимость(на первом месте, 92.6%) или на свадьбу(на втором месте, 92%), нежели чем кредит связанный с автомобилем или образованием(последнее место, 90.5%)

Также нужно обратить внимание на заполнение следующих столбцов с данными: прежде всего, это стаж клиента (в каких единицах измерения считается и с каким знаком) и его доход, а также более аккуратно заполнять ячейки с возрастом (0 лет), количеством детей(-1 и 20) и полом(XNA)